# Visual Attention Models for Medical Image Analysis

#### Anton Karakochev (s0553324@htw-berlin.de)

## 0. Install dependencies using pip 

In [ ]:
!pip install torch torchvision torchviz matplotlib numpy tqdm scikit-image

## 1. Dataset creation

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import Normal
from torch.utils.data import Dataset
from torchviz import make_dot

import skimage.measure

import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np


import unittest
import time
import shutil
import os
import math

from tqdm.notebook import tqdm

In [ ]:
#parameters, just as global variables for convenience

# glimpse sensor
global_glimpse_sensor_n = 3
global_glimpse_senor_patch_size = 12
global_glimpse_sensor_scale = 2

# glimpse network
global_glimpse_network_output_size = 28
global_glimpse_network_hidden_layer_size = 32

# core network
global_lstm_hidden_size = 28
global_lstm_num_layers = 2

# location network
global_std_dev = 0.18

# action network
global_num_classes = 10

# training
global_num_epochs = 500
global_batch_size = 50
global_learning_rate = 0.001
global_model_patience = 100

# dataset
#note that when using 'augmented-medical the number of output classes is 4'
#one of 'mnist', 'augmented', 'transformed', 'augmented-medical'
global_dataset_name = 'transformed'

global_num_glimpses = 8

In [ ]:
# model name, used for storage and checkpointing
global_model_name = f"{global_dataset_name}-{global_num_epochs}-{global_glimpse_sensor_n}-{global_glimpse_senor_patch_size}x{global_glimpse_senor_patch_size}"

In [ ]:
#if gpu is available, use it 
global_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#set seed for reproducibility
torch.manual_seed(1)
np.random.seed(1)

In [ ]:
def combine_images(images, output_dim):
    """
    Combines the given images into a single image of output_dim size. Combinations are done randomly and 
    overlapping is possible. Images will always be within bounds completely.
    """
    np_images = np.array(images)
    input_dim = np_images.shape[-1]
    new_image = np.zeros(shape=(output_dim,output_dim), dtype = np.float32)
    for image in np_images:
        i, j = np.random.randint(0, output_dim  - input_dim, size = 2)
        new_image[i:i+input_dim, j:j+input_dim] = image
    return new_image

class AugmentedMedicalMNISTDataset(Dataset):
    """
    Augmented mnist meant to mimic whole-slide-images of tumor cells.
    9's represent cancer cells. There are 4 different labels, based on the number of 9's:
    
    zero 9's          - no cancer
    one 9             - isolated tumor cell
    two 9's           - micro-metastasis 
    three or more 9's - macro-metastasis
    
    Each image contains between 3 and 10 cells at random, which may be overlapping.
    It consists of 5000 items of each category(total 20.000) for training and 500(2.000) of each for testing
    of size 256 x 256. 
    """

    def __init__(self, 
                 root_dir,
                 train,
                 mnist_transform = None,
                 transform = None, 
                 total_train = 20000,
                 total_test = 2000,
                 n_partitions_test = 1,
                 n_partitions_train = 5):
        
        self.mnist_transform = mnist_transform
        self.root_dir = root_dir
        self.train = train
        self.total = total_train if self.train else total_test
        self.n_partitions_test  = n_partitions_test
        self.n_partitions_train = n_partitions_train
        self.dir = os.path.join(root_dir,"MEDNIST", "train" if train else "test")
        self.transform = transform
        
        self.__create_dataset_if_needed()
                
        self.__load_data()
        
    def __dataset_exists(self):
        # mkdir if not exists
        os.makedirs(self.dir, exist_ok = True)
        len_files = len(os.listdir(self.dir)) 
        if len_files > 0:
            print("Data existing, skipping creation.")
            return True
        else:
            print("Dataset missing. Creating...")
        return False
            
    
    def __get_cell_counts(self, items_per_class_count, class_index):
        # exclusive
        max_items = 11
        min_number_of_cells = 3
        # 0,1,2,3+ for no tumor cells, isolated tumor cells, 
        # micro-metastasis and macro-metastasis respectively
        num_tumor_cells = class_index if class_index != 3 else np.random.randint(3, max_items) 

        num_healthy_cells = max_items - num_tumor_cells
        if num_healthy_cells + num_tumor_cells < min_number_of_cells:
            num_healthy_cells = min_number_of_cells - num_tumor_cells

        return (num_tumor_cells, num_healthy_cells)
            
    def __generate_for_class(self,
                                   items,
                                   items_per_class_count,
                                   class_index,
                                   uid,
                                   all_tumor_cell_images,
                                   all_healthy_cell_images):
        for _ in range(items_per_class_count):
            num_tumors, num_healthy = self.__get_cell_counts(items_per_class_count, class_index)

            healthy_idxs = np.random.randint(0,len(all_healthy_cell_images), num_healthy)
            tumor_idxs   = np.random.randint(0,len(all_tumor_cell_images), num_tumors)

            cells = np.vstack((all_healthy_cell_images[healthy_idxs], all_tumor_cell_images[tumor_idxs]))
            image = combine_images(cells, 256)
            image = np.expand_dims(image, axis = 0)
            self.data.append(image)
            self.labels.append(class_index)
            uid += 1
        return uid
            
    def __create_dataset_if_needed(self):
        if self.__dataset_exists():
            return 
        
        self.data = []
        self.labels = []
        
        # in how many partitions to split dataset creation
        partitions_count = 10
        
        # number of classes in output (fixed)
        num_classes = 4
        
        mnist = torchvision.datasets.MNIST(root ='./data',
                                           train = self.train,
                                           download = True,
                                           transform = self.mnist_transform)
        
        mnist_loader = iter(torch.utils.data.DataLoader(mnist, 
                                                        batch_size = int(self.total/partitions_count), 
                                                        shuffle = True, 
                                                        num_workers = 0))
        uid = 0
        
        for _ in range(partitions_count):
            batch, mnist_labels = mnist_loader.next()
            # 9's represent tumors
            all_tumor_cell_images = batch[mnist_labels == 9]
            # everything else except 6's healthy cells
            all_healthy_cell_images = batch[(mnist_labels != 9) & (mnist_labels != 6)]
            
            items_per_class_count = int(self.total/(num_classes * partitions_count))
            
            for class_index in range(num_classes):
                    uid = self.__generate_for_class(class_index, 
                                                  items_per_class_count,
                                                  class_index,
                                                  uid,
                                                  all_tumor_cell_images,
                                                  all_healthy_cell_images)
        self.__store()
        print("Done.")
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, uid):
        if torch.is_tensor(uid):
            uid = uid.tolist()
        label = self.labels[uid]
        sample = self.data[uid]
        if self.transform:
            sample = self.transform(sample)

        return (sample,label)

    def __store(self):
        n_partitions = self.n_partitions_train if self.train else self.n_partitions_test
        
        assert(len(self.data) == len(self.labels))
        max_index = len(self.data)
        partition_size = max_index/n_partitions
        for i in range(n_partitions):
            start,end =(int(partition_size * i), int(partition_size * (i+1)))
            partition = np.array(self.data[start:end])
            np.save(os.path.join(self.dir, "part_" + str(i)), partition)
        
        np.save(os.path.join(self.dir, "labels"), np.array(self.labels))
    
    def __load_data(self):
        n_partitions = self.n_partitions_train if self.train else self.n_partitions_test
        data = []
        for i in range(n_partitions):
            data.append(np.load(os.path.join(self.dir, "part_" + str(i)+".npy")))
        self.data = np.vstack(data)
        self.labels = np.load(os.path.join(self.dir, "labels.npy"))

In [ ]:
def transformed_mnist_transformation():
    def f(image):
        np_image = np.array(image)
        input_dim = np_image.shape[-1]
        new_image = np.zeros(shape=(60,60), dtype = np.float32)
        i, j = np.random.randint(0, 60  - input_dim, size = 2)
        new_image[i:i+input_dim, j:j+input_dim] = np_image
        return new_image
    return transforms.Compose(
        [torchvision.transforms.Lambda(f),
        torchvision.transforms.ToTensor()])

def augmented_mnist_transformation():
    return transforms.Compose([
        torchvision.transforms.RandomAffine(degrees = (-180,180),scale = (0.5,1.0),),
        torchvision.transforms.ToTensor()])

def dataset(train = True,
            name = global_dataset_name):
    if name == 'mnist':
        transform = torchvision.transforms.ToTensor()
    elif name == 'augmented':
        transform = augmented_mnist_transformation()
    elif name == 'transformed':
        transform = transformed_mnist_transformation()
    elif name == 'augmented-medical' :
        return AugmentedMedicalMNISTDataset(root_dir='.', train = train,mnist_transform = augmented_mnist_transformation())
    else:
        raise Exception("unrecognized dataset name.")
    return torchvision.datasets.MNIST(root='./data', train = train , download = True, transform = transform)

In [ ]:
mnist_train   = dataset()
train_loader  = torch.utils.data.DataLoader(mnist_train, batch_size=4,
                                          shuffle=True, num_workers=2)
images,labels = iter(train_loader).next()

In [ ]:
def show(images,labels = None):
    if labels == None:
        labels = [" "] * len(images)
    elif isinstance(labels,torch.Tensor): 
        labels = [label.item() for label in labels]
    fig, axes = plt.subplots(1, len(images), figsize=(25,25))
    for (a,image,label) in zip(axes.ravel(),images,labels):
        a.imshow(image[0].numpy(), cmap = plt.cm.gray)
        a.set_title(label, fontsize=20)

    fig.tight_layout()
show(images,labels)

# 2. Neural Network

### 2.1 Glimpse Sensor

In [ ]:
def normalized_position(positions, dim):
    """ 
    Given positions `positions` [0, dim] and image dimension `dim`,
    converts them to normalized positions [-1, -1]
    
    Args:
    ----
    positions: 2D Tensor (B x 2)
    dim: Int
    """
    res = (2 * positions / float(dim)) - 1
    return torch.clamp(res,-1,1)

def inverse_normalized_position(norm_positions, dim):
    """ 
    Given normalized positions `norm_positions` [-1,-1] and image dimension `dim`,
    converts them to positions within the image [0, dim]
    
    Args:
    ----
    norm_positions: 2D Tensor (B x 2)
    dim: Int
    """
    res = (((norm_positions + 1) * float(dim)) / 2).int()
    return torch.clamp(res,0,dim)

In [ ]:
class GlimpseSensor:
    """
    Extracts retina-like representations of given images.
    """
    def __init__(self, n, scale, patch_size):
        """
        Args:
        ----
        n: Int          - number of patches to extract
        patch_size: Int - size of a patch
        scale: Int      - scaling factor for each lower resolution 
        """
        self.n = n
        self.scale = scale
        self.patch_size = patch_size
    
    def extract_glimpses(self, images, location):
        """
        Extracts a retina like representation of the given images `x`, 
        centered at `location` and containing `n` resolution patches of size (`patch_size`, `patch_size`), 
        where each has a `scale` times lower resolution than the previous.
        
        Currently images of only 1 Channel (greyscale) can be processed, 
        but extension to coloured images is not hard.
        
        Args:
        ----
        x: 4D Tensor (B x C x H x W) - The image mini batch
        location: 2D Tensor (B x 2)  - normalized positions in images
        
        Returns
        -------
        4D Tensor (B x n x scale_size x scale_size ) - extracted glimpses
        """
        
        B, C, H ,W = images.shape
        l = inverse_normalized_position(location, W)
        patches = []
        for patch_index in range(self.n):
            current_scale = self.scale ** patch_index
            size = self.patch_size * current_scale
            patch = self.extract_patch(images, l , size)
            patches.append(patch)
        
        # for each patch in patches make sure size is equal to patch_size
        patches_square = []
        for patch in patches:
            size = patch.shape[-1]
            stride = int(size / self.patch_size)
            squared = F.avg_pool2d(patch, stride)
            patches_square.append(squared)
        # stack and bring in desired dimensions
        return torch.stack(patches_square).squeeze().transpose(0,1)
        
    def extract_patch(self, images, l , size):
        # shift image by half-size, effectively making centered l coordinates into top left coordinates
        # and at the same time handling out of bounds problems
        half_size = math.ceil (size / 2)
        images = F.pad(images,(half_size, half_size, half_size, half_size))
        
        # x,y tuples
        start = l
        # shifted x,y tuples
        end = start + size
        
        n_images = images.shape[0]
        patch = []
        for i in range(n_images):
            patch_i = images[i, :, start[i, 1] : end[i, 1], start[i, 0] : end[i, 0]]
            patch.append(patch_i)
        return torch.stack(patch)

In [ ]:
# plot some images by looking at the center
gs = GlimpseSensor(global_glimpse_sensor_n, global_glimpse_sensor_scale, global_glimpse_senor_patch_size)
glimpses = gs.extract_glimpses(images,torch.zeros((images.shape[0],2)))

for i in range(images.shape[0]):
    scales = [(gs.scale ** n_i) * gs.patch_size for n_i in range(gs.n)]
    l = [f"class: {labels[i].item()}, {images[i].shape[1]} x {images[i].shape[2]}"] + [f"{scale} x {scale}" for scale in scales]
    to_show = [images[i]] + [g.unsqueeze(0) for g in glimpses[i,:,:,:]]
    show(to_show,l)

In [ ]:
p = torch.tensor([[ 1.0633, -0.0473],
        [-0.0921,  0.0515],
        [ 0.1845,  0.1011]])
gs.extract_glimpses(images[0:2],p[0:2])

### 2.2 Glimpse Network

In [ ]:
class GlimpseNetwork(nn.Module):
    """
    Takes a location and encodes a `where` and `what` feature representation, which are combined together. 
    Consists of a combination of convolutional, max pooling and fully connected layers.
    """
    def __init__(self,
                 n, 
                 patch_size, 
                 scale,
                 glimpse_output_size,
                 hidden_layer_size
                ):
        """
        Args:
        ----
        n: Int                   - number of patches to extract
        patch_size: Int          - size of a patch
        scale: Int               - scaling factor for each lower resolution 
        glimpse_output_size: Int - size of output of last fully connected layer
        hidden_layer_size: Int   - size of hidden layer (number of units)
        """
        super(GlimpseNetwork, self).__init__()
        self.glimpse_sensor = GlimpseSensor(n = n, scale = scale, patch_size = patch_size)
        self.glimpse_output_size = glimpse_output_size
        self.hidden_layer_size = hidden_layer_size
                
        # what 
        
        # padding of 1, to ensure same dimensions
        self.conv1 = nn.Conv2d(in_channels = self.glimpse_sensor.n, out_channels = 16, kernel_size = 3, padding = 1)
        
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = 3, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = 16, track_running_stats = True)
         
        self.max_pool1 = nn.MaxPool2d(kernel_size = 3, stride = 3, padding = 1)
        
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)

        self.max_pool2 = nn.MaxPool2d(kernel_size = 3, stride = 3, padding = 1)
        
        # TODO size in and out
        # W * H of previous layer * depth
        self.fc1 = nn.Linear(in_features = 128, out_features = self.glimpse_output_size)
        self.bn2 = nn.BatchNorm1d(num_features = self.glimpse_output_size, track_running_stats = True)
        
        # where
        # in_features = 2, loc is a tuple of (x,y)
        self.loc_fc1 = nn.Linear(in_features = 2, out_features = self.hidden_layer_size)
        self.loc_fc2 = nn.Linear(in_features = self.hidden_layer_size, out_features = self.glimpse_output_size)
        
    def forward(self, x, location):
        glimpses = self.glimpse_sensor.extract_glimpses(x, location)
        
        # what
        # 3 conv layers
        h = self.conv1(glimpses)
        h = F.relu(h)
        
        h = self.conv2(h)
        h = self.bn1(h)
        h = F.relu(h)
        
        h = self.max_pool1(h)
        
        h = F.relu(self.conv3(h))
        
        h = self.max_pool2(h) 
        
        # flatten
        # keep batch dimension and determine other one automatically
        h = h.view(x.shape[0],-1)
        
        # fully connected layers
        h = self.fc1(h)
        h = self.bn2(h)
        h = F.relu(h)
        
        # where
        l = self.loc_fc1(location)
        l = F.relu(l)
        l = self.loc_fc2(l)
        
        # combine what and where
        g = F.relu(h * l)
        return g

### 2.3 Core Network


In [ ]:
class CoreNetwork(nn.Module):
    """
    Takes the `what` and `where` feature representation by the Glimpse Network
    and a hidden state to produce a new hidden state and prediction.
    Consists of 2 LSTM layers stacked on top of each other.
    """
    def __init__(self,
                 batch_size,
                 lstm_hidden_size,
                 glimpse_input_size,
                 num_layers):
        """
        Args:
        batch_size:Int          - size of a batch. Used compute the inital hidden state 
        glimpse_input_size: Int - size of the input. Equal to the output size of the Glimpse Network 
        lstm_hidden_size: Int   - size of the output and hidden layers
        num_layers: Int         - number of stacked LSTM layers
        ----
        """
        super(CoreNetwork, self).__init__()
        self.glimpse_input_size = glimpse_input_size
        self.lstm_hidden_size = lstm_hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size 
        # batch_first = false -> (SEQ x B x Features)
        self.stacked_lstm = nn.LSTM(input_size = self.glimpse_input_size,
                                    hidden_size = self.lstm_hidden_size,
                                    num_layers = self.num_layers,
                                    batch_first = False)
    
    def forward(self, x, hidden):
        # does one step 
        (prediction, new_hidden) = self.stacked_lstm(x, hidden)
        # new_hidden is a tuple of layer state and cell state 
        # remove channel dimension
        return (prediction[0], new_hidden)
        
    def initial_hidden_state(self):
        """Initialize hidden state and cell state with zeroes."""
        return (torch.zeros(self.num_layers, self.batch_size, self.lstm_hidden_size).to(global_device),
                torch.zeros(self.num_layers, self.batch_size, self.lstm_hidden_size).to(global_device))

### 2.4 Location Network

In [ ]:
class LocationNetwork(nn.Module):
    """
    Takes the hidden state of the Core Network and produces a next location to attend to. 
    The next location is determined stochastically from a normal distribution 
    with a fixed deviation and a learned mean.
    
    Outputs the next location alongside the mean.
    
    Non-differentiable and needs to be trained with Reinforcment Learning.
    """
    def __init__(self,
                input_size,
                std_dev):
        """
        Args:
        ----
        input_size: Int - the input size. Equal to the output size of the Core Network
        std_dev: Float  - the fixed standard deviation/variance 
        """
        super(LocationNetwork, self).__init__()
        # location -> (x, y)
        self.fc = nn.Linear(input_size, 2)
        self.std_dev = std_dev
        
    def forward(self, h_t):
        #predicts the mean [-1,1] based on the network
        mean = torch.tanh(self.fc(h_t.detach()))
                                  
        normal_dist = Normal(mean, torch.tensor((self.std_dev,self.std_dev)).to(global_device))
        
        next_location = normal_dist.sample()
        log_probs = normal_dist.log_prob(next_location)       
        log_probs = torch.sum(log_probs, dim = 1)
        return (next_location, log_probs)            

In [ ]:
class BaselineNetwork(nn.Module):
    """
    Incorporates the baseline in the reward function
    to reduce the variance of the gradient update.
    """
    def __init__(self, 
                 input_size, 
                 output_size):
        super(BaselineNetwork, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, hidden_t):
        baseline_t = F.relu(self.fc(hidden_t.detach()))
        return baseline_t

### 2.5 Action Network

In [ ]:
class ActionNetwork(nn.Module):
    """
    Takes the internal state of the Core Network to produce the final output classification.
    """
    def __init__(self,
                 input_size,
                 num_classes):
        super(ActionNetwork, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
        
    def forward(self, hidden_t_bottom_layer):
        predictions = F.log_softmax(self.fc(hidden_t_bottom_layer), dim = 1)
        return predictions

### 2.6 Deep Recurrent Attention Model


In [ ]:
class DeepRecurrentAttentionModel(nn.Module):
    """
    Combines all networks described above.
    """
    def __init__(self,
                 glimpse_sensor_n = global_glimpse_sensor_n, 
                 glimpse_senor_patch_size = global_glimpse_senor_patch_size, 
                 glimpse_sensor_scale = global_glimpse_sensor_scale,
                 glimpse_network_output_size = global_glimpse_network_output_size,
                 glimpse_network_hidden_layer_size = global_glimpse_network_hidden_layer_size,
                 lstm_hidden_size = global_lstm_hidden_size,
                 lstm_num_layers = global_lstm_num_layers,
                 number_of_glimpses = global_num_glimpses,
                 batch_size = global_batch_size,
                 num_classes = global_num_classes,
                 std_dev = global_std_dev
                ):
        super(DeepRecurrentAttentionModel, self).__init__()
        self.batch_size = batch_size
        self.std_dev = std_dev
        self.number_of_glimpses = number_of_glimpses
        
        self.glimpse_network = GlimpseNetwork(n = glimpse_sensor_n,
                                              patch_size = glimpse_senor_patch_size,
                                              scale = glimpse_sensor_scale,
                                              glimpse_output_size = glimpse_network_output_size,
                                              hidden_layer_size = glimpse_network_hidden_layer_size)
        
        self.core_network = CoreNetwork(lstm_hidden_size = lstm_hidden_size,
                                       glimpse_input_size = glimpse_network_output_size,
                                       num_layers = lstm_num_layers,
                                       batch_size = batch_size)
        
        self.location_network = LocationNetwork(input_size = lstm_hidden_size, std_dev = std_dev)
        
        self.action_network = ActionNetwork(input_size = lstm_hidden_size, num_classes = num_classes)
        
        self.baseline_network = BaselineNetwork(input_size = lstm_hidden_size, output_size = 1)
    
    def initial_hidden_state(self):
        return self.core_network.initial_hidden_state()
    
    def initial_glimpse_location(self):
        return torch.Tensor(self.batch_size, 2).fill_(0.0).to(global_device)
    
    def forward(self,
                x,
                prev_hidden,
                prev_location,
                also_compute_probabilities = False):
        
        g_t = self.glimpse_network(x, prev_location.detach())
        # add seq dimension
        g_t = g_t.unsqueeze(0)
        
        # hidden_t - top layer output
        # total_hidden_t - all layers including cell parameters
        (hidden_t, total_hidden_t) = self.core_network(g_t, prev_hidden)
        
        (location_t, location_mean_t) = self.location_network(hidden_t.detach())
        baseline_t = self.baseline_network(hidden_t.detach())
        
        if also_compute_probabilities:
            # takes not the top, but the bottom layer in the paper
            probs_t = self.action_network(total_hidden_t[0][0])
            return (total_hidden_t, location_t, location_mean_t, baseline_t, probs_t)
        else:
            return (total_hidden_t, location_t, location_mean_t, baseline_t)

## 3. Train

In [ ]:
class RunningAverage(object):
    """Computes a running average."""
    def __init__(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def add(self, val, n = 1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class Train:
    """
    Entry class, takes care of training, checkpoints and output.
    """
    def __init__(self,
                 train_loader,
                 valid_loader,
                 test_loader,
                 model_name = global_model_name, 
                 learning_rate = global_learning_rate,
                 num_epochs = global_num_epochs,
                 batch_size = global_batch_size,
                 num_glimpses = global_num_glimpses,
                 checkpoint_dir = "./models",
                 model_patience = global_model_patience ,
                 resume = True
                 ):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.num_glimpses = num_glimpses
        self.batch_size = batch_size
        self.model_name = model_name
        self.checkpoint_dir = checkpoint_dir
        self.best_valid_acc = 0
        self.start_epoch = 0
        self.resume = resume
        self.model_patience = model_patience
        self.current_without_improvement = 0
        
        # Device configuration
        self.device = global_device
        
        # loaders
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader

        self.num_train = len(train_loader.dataset)
        self.num_valid = len(valid_loader.dataset)
        self.num_test = len(test_loader.dataset)
        
        self.model = DeepRecurrentAttentionModel().to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.MSELoss = nn.MSELoss()
        self.NLLLoss = nn.NLLLoss()
        
    def train(self):
        if self.resume:
            self.load_checkpoint(load_best = False)
        for epoch in range(self.start_epoch, self.num_epochs):
            # switch layers to train mode
            self.model.train()
            # train for 1 epoch
            train_loss, train_acc = self.train_single_epoch()            
            print("Epoch: {}, avg. loss: {:3.3f}, avg. acc: {:3.3f}".format((epoch), train_loss, train_acc))
            
            # evaluate on validation set
            valid_loss, valid_acc = self.evaluate(is_test = False, epoch = epoch)

            # update best
            is_best = valid_acc >  self.best_valid_acc
            if is_best:
                self.current_without_improvement = 0
            else:
                self.current_without_improvement += 1
            self.best_valid_acc = max(valid_acc, self.best_valid_acc)
            
            # checkpoint
            self.save_checkpoint(
                {'epoch': epoch + 1,
                 'model_state': self.model.state_dict(),
                 'optim_state': self.optimizer.state_dict(),
                 'best_valid_acc': self.best_valid_acc,
                 },is_best
            )
            if self.current_without_improvement > self.model_patience:
                print(f"{self.model_patience} epochs without improvement. Stopping.")
                return
    
    def forward_pass(self, images, labels):
        hidden_t = self.model.initial_hidden_state()
        location_t = self.model.initial_glimpse_location()
        baselines = []
        log_probs = []
        for glimpse_i in range(self.num_glimpses):
            is_last = glimpse_i == self.num_glimpses - 1
            if is_last :
                (hidden_t, location_t, log_probs_t, baseline_t, probs_t) = self.model(images,
                                                                                  hidden_t,
                                                                                  location_t, 
                                                                                  also_compute_probabilities = True)
            else:
                (hidden_t, location_t, log_probs_t, baseline_t) = self.model(images, hidden_t, location_t)
            baselines.append(baseline_t[:,0])
            
            log_probs.append(log_probs_t)
 
        baselines = torch.stack(baselines).transpose(1, 0)
        log_probs = torch.stack(log_probs).transpose(1, 0)
         
        # compute loss and rewards

        # reward
        predicted = probs_t.argmax(dim = 1).detach()
        rewards = (predicted == labels).float()

        rewards = rewards.unsqueeze(1).repeat(1, self.num_glimpses)

        # losses 
        # action
        loss_action = self.NLLLoss(probs_t, labels)

        # baseline
        loss_baseline = self.MSELoss(baselines, rewards)

        # reinforcement
        adjusted_reward = rewards - baselines.detach()
        
        #minus because the higher the reward: the better
        log_likelihood_rewards = torch.sum( - log_probs * adjusted_reward, dim = 1)
        loss_reinforce = torch.mean(log_likelihood_rewards, dim = 0)

        loss = loss_action + loss_baseline + loss_reinforce
        
        #if np.random.randint(1,51,1).item() == 50:
        #    print(f"Loss = [A:{loss_action:+7.3},B:{loss_baseline:+7.3},R:{loss_reinforce:+7.3}] - Weight sum = [A:{self.model.action_network.fc.weight.detach().sum():+7.3},B:{self.model.baseline_network.fc.weight.detach().sum():+7.3},R:{self.model.location_network.fc.weight.detach().sum():+7.3}]")
        
        return (predicted, loss)
    
    def train_single_epoch(self):
        epoch_losses = RunningAverage()
        epoch_accuracies = RunningAverage()
        
        with tqdm(total = self.num_train) as pbar:
            start = time.time()
            # images - mini batch
            for i, (images, labels) in enumerate(self.train_loader): 
                images = images.to(self.device)
                labels = labels.to(self.device)
                # forward pass
                (predictions,loss) = self.forward_pass(images, labels)
                # backward and optimize
                #print(self.model.action_network.fc.weight.grad)
                # zero out current gradiate accumulators
                self.optimizer.zero_grad()
                # propagate new ones
                loss.backward()
                
                #print(self.model.baseline_network.fc.weight.grad)    
                self.optimizer.step()
                
                # compute accuracy
                correct = (predictions == labels).float()
                acc = 100 * (correct.sum() / len(labels))
                
                elapsed = time.time() - start
                # store
                epoch_losses.add(loss.item(), len(images))
                epoch_accuracies.add(acc.item(), len(images))
                
                pbar.set_description(("{:.1f}s loss: {:.3f} , acc: {:3.3f}"
                                      .format(elapsed, loss.item(), acc.item())))
                pbar.update(self.batch_size)
        return (epoch_losses.avg, epoch_accuracies.avg)
        
    def evaluate(self, is_test, epoch = None):
        loader = self.test_loader if is_test else self.valid_loader
        name = "test" if is_test else f"Epoch: {epoch}, validation"
        
        n = self.num_test if is_test else self.num_valid
        if is_test:
            self.load_checkpoint(load_best = True)
        
        # switch layers to evaluation mode    
        self.model.eval()
        
        correct = 0
        losses = RunningAverage()
        
        with torch.no_grad():
            #with tqdm(total = self.num_train) as pbar:
            start = time.time()
            # images - mini batch
            for i, (images, labels) in enumerate(loader):
                images = images.to(self.device)
                labels = labels.to(self.device)
                # forward pass
                (predictions, loss) = self.forward_pass(images, labels)
                
                # compute accuracy
                correct += (predictions == labels).int().sum()
                losses.add(loss.item(), len(images))
                elapsed = time.time() - start
                
                # store
                losses.add(loss.item(), len(images))

                #pbar.set_description(("{:.1f}s Evaluating... ".format(elapsed)))
                #pbar.update(self.batch_size)
        accuracy = (100. * correct) / n
        print(f'[*] {name} - {correct}/{n} Correct, Acc: ({accuracy:3.2f}%), Err({(100-accuracy):3.2f}%)')
        return (losses.avg, accuracy)
    
    def save_checkpoint(self, state, is_best):
        filename = self.model_name + '_ckpt'
        ckpt_path = os.path.join(self.checkpoint_dir, filename)
        os.makedirs(self.checkpoint_dir,exist_ok = True)
        torch.save(state, ckpt_path)
                    
        if is_best:
            filename = self.model_name + '_model_best'
            shutil.copyfile(ckpt_path, os.path.join(self.checkpoint_dir, filename))

    def load_checkpoint(self, load_best):
        print("[*] Loading model from {}".format(self.checkpoint_dir))

        filename = self.model_name + '_ckpt'
        if load_best:
            filename = self.model_name + '_model_best'
        ckpt_path = os.path.join(self.checkpoint_dir, filename)
        if  not os.path.exists(ckpt_path):
            print("No checkpoint found, starting from scratch")
            return
        ckpt = torch.load(ckpt_path)

        # load variables from checkpoint
        self.best_valid_acc = ckpt['best_valid_acc']
        #self.start_epoch = ckpt['epoch']
        self.model.load_state_dict(ckpt['model_state'])
        self.optimizer.load_state_dict(ckpt['optim_state'])

        if load_best:
            print(
                "[*] Loaded {} checkpoint @ epoch {} "
                "with best valid acc of {:.3f}".format(
                    filename, ckpt['epoch'], ckpt['best_valid_acc'])
            )
        else:
            print(
                "[*] Loaded {} checkpoint @ epoch {}".format(
                    filename, ckpt['epoch'])
            )

In [ ]:
mnist_train_data = dataset()
mnist_train_length = int(len(mnist_train_data) * 0.9)
mnist_valid_length = len(mnist_train_data) - mnist_train_length

(mnist_train, mnist_valid) = torch.utils.data.random_split(mnist_train_data,(mnist_train_length, mnist_valid_length))
mnist_test = dataset(train = False)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = global_batch_size,
                                          shuffle = True, num_workers = 2)

valid_loader = torch.utils.data.DataLoader(mnist_valid, batch_size = global_batch_size,
                                          shuffle = False, num_workers = 2)

test_loader = torch.utils.data.DataLoader(mnist_valid, batch_size = global_batch_size,
                                          shuffle = False, num_workers = 2)

trainer = Train(train_loader = train_loader,
      valid_loader = valid_loader,
      test_loader = test_loader)

### 3.1 Plot backward graph

### 3.2 Start training

In [ ]:
trainer.train()

## 4. Evaluate

trainer.evaluate(is_test = True)

## 5. Plot glimpses

In [ ]:
plot_images, plot_labels = iter(train_loader).next()

## 6. Unit Tests

In [ ]:
class TestNormalizedPositions(unittest.TestCase):
    
    def assertTorchEqual(self,a , b):
        return self.assertTrue(torch.all(a.eq(b)))
    
    def test_normalized_position_center(self):
        actual = normalized_position(torch.tensor((50,50)),100)
        expected = torch.tensor((0.0,0.0))
        self.assertTorchEqual(expected,actual)
        
    def test_normalized_position_top_left(self):
        actual = normalized_position(torch.tensor((0,0)),100)
        expected = torch.tensor((-1,-1))
        self.assertTorchEqual(expected,actual)
        
    def test_normalized_position_bottom_right(self):
        actual = normalized_position(torch.tensor((100,100)), 100)
        expected = torch.tensor((1,1))
        self.assertTorchEqual(expected,actual)
        
    def test_normalized_position_other_location(self):
        actual = normalized_position(torch.tensor((25,75)),100)
        expected = torch.tensor((-0.5,0.5))
        self.assertTorchEqual(expected,actual)
    
    def test_normalized_position_out_of_bound(self):
        actual = normalized_position(torch.tensor((-10,120)),100)
        expected = torch.tensor((-1,1))
        self.assertTorchEqual(expected,actual)
        
    def test_normalized_position_multiple(self):
        actual = normalized_position(torch.tensor(((25,75),(75,75),(50,50))),100)
        expected = torch.tensor(((-0.5,0.5),(0.5,0.5),(0.0,0.0)))
        self.assertTorchEqual(expected,actual)
        
    #inverse
    
    def test_inverse_normalized_position_center(self):
        actual = inverse_normalized_position(torch.tensor((0.0,0.0)),100)
        expected = torch.tensor((50,50))
        self.assertTorchEqual(expected,actual)
        
    def test_inverse_normalized_position_top_left(self):
        actual = inverse_normalized_position(torch.tensor((-1,-1)),100)
        expected = torch.tensor((0,0))
        self.assertTorchEqual(expected,actual)
        
    def test_inverse_normalized_position_bottom_right(self):
        actual = inverse_normalized_position(torch.tensor((1,1)), 100)
        expected = torch.tensor((100,100))
        self.assertTorchEqual(expected,actual)
        
    def test_inverse_normalized_position_other_location(self):
        actual = inverse_normalized_position(torch.tensor((-0.5,0.5)),100)
        expected = torch.tensor((25,75))
        self.assertTorchEqual(expected,actual)
    
    def test_inverse_normalized_position_out_of_bounds(self):
        actual = inverse_normalized_position(torch.tensor((-1.2,1.2)),100)
        expected = torch.tensor((0,100))
        self.assertTorchEqual(expected,actual)

    def test_inverse_normalized_position_multiple(self):
        actual = inverse_normalized_position(torch.tensor(((-0.5,0.5),(0.5,0.5),(0.0,0.0))),100)
        expected = torch.tensor(((25,75),(75,75),(50,50)))
        self.assertTorchEqual(expected,actual)

In [ ]:
import scipy.misc
gs = GlimpseSensor(4, 2, 50)
im = scipy.misc.face(True).astype(int)[100:600,300:800]
im_racoon = torch.tensor([im,im]).unsqueeze(1)

In [ ]:
plt.imshow(im,cmap=plt.cm.gray)

In [ ]:
glimpses = gs.extract_glimpses(im_racoon, torch.tensor(((0.0,0.0),(0.0,0.0))))
show([g.unsqueeze(0) for g in glimpses[0]],["CENTER","","",""])

In [ ]:
glimpses = gs.extract_glimpses(im_racoon, torch.tensor(((-1,-1),(-1,-1))))
show([g.unsqueeze(0) for g in glimpses[0]],["TOP-LEFT","","",""])

In [ ]:
glimpses = gs.extract_glimpses(im_racoon, torch.tensor(((-1,0),(-1,0))))
show([g.unsqueeze(0) for g in glimpses[0]],["LEFT","","",""])

In [ ]:
glimpses = gs.extract_glimpses(im_racoon, torch.tensor(((-0.5,0.5),(-0.5,0.5))))
show([g.unsqueeze(0) for g in glimpses[0]],["offset from center","","",""])